In [1]:
from pathlib import Path 
import os, dotenv
dotenv.load_dotenv()
os.chdir(Path(os.getenv("PYTHONPATH")).expanduser())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [3]:
preformat_data_df = pd.read_json("data/training_datasets/train_test_split/all_hybrid.jsonl", lines=True)
preformat_data_df['filetype'] = preformat_data_df['filetype'].apply(lambda x: 'cryptol' if x == 'cry' else 'saw')
#preformat_data_df = preformat_data_df[preformat_data_df["filetype"] == "saw"].reset_index(drop=True)
preformat_data_df.head()


,filename,filetype,content,variant,set
0,cryptol/examples/splitAt.cry,cryptol,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",hybrid,unsupervised
1,cryptol/examples/AE.cry,cryptol,/*\nImplementation of the algorithms from the ...,hybrid,unsupervised
2,cryptol/examples/xor_cipher.cry,cryptol,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,hybrid,supervised
3,cryptol/examples/zero_weird.cry,cryptol,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,hybrid,unsupervised
4,cryptol/examples/builtin_lifting.cry,cryptol,"//builtins lift over tuples, seqs, and records...",hybrid,unsupervised


In [4]:
import src.preprocessing.sft_saw as sft_saw

ROOT_DIR = "~/SecurityAnalytics"

java_test = preformat_data_df[preformat_data_df["filename"] == "saw-script/examples/java/branchsat.saw"].reset_index(drop=True)
#print(java_test)

saw_path = f"{ROOT_DIR}/{java_test.iloc[0]['filename']}"
#print(saw_path)
associated_sources = sft_saw.get_associated_sources(
    saw_path,
    java_test.iloc[0]['content']
)
for source in associated_sources:
    print(f"--- {source} ---")

user_prompt, input_query, _ = sft_saw.build_user_prompt(
    filename=java_test.iloc[0]['filename'],
    content=java_test.iloc[0]['content'],
)
print(user_prompt)
print(f'DEBUG MESSAGE\n{input_query}\nEND DEBUG MESSAGE')

--- {'BranchTest.java': '```java\nclass BranchTest {\n    static int f(int x) {\n        int r = 0;\n        for(int i = 0; i < x; i++) {\n            r = r + i;\n        }\n        return r;\n    }\n}\n\n```'} ---
Primary SAW file: saw-script/examples/java/branchsat.saw
Associated source files: BranchTest.java
Goal: Produce one *instruction* telling an assistant to write a SAWScript verification harness that directly references functions/methods defined in the associated C/Java source in the associated source files.
Rules for the instruction you will produce:
- SAW only. Infer the backend from the excerpt: use jvm_* if `java_load_class` appears; use llvm_* if `llvm_load_module` appears.
- Require a let-bound spec, e.g., `let <name>_spec = do { ... };` (do NOT pass an inline `do { ... }` directly as an argument).
- Reference exact symbol names/signatures from the associated source (e.g., class.method or function prototypes) and call them accordingly.
- Prefer symbolic inputs (`jvm_fres

In [5]:


from src.preprocessing.sft_instruct_preprocess import iter_call_openai_structured, alpaca_df_to_qwen_messages
file_cache_path = "cache/alpaca_instruct_cache/SFT_singular_files.jsonl"

test_df = preformat_data_df.sample(10, random_state=42).reset_index(drop=True)
input_mode="full"

result = iter_call_openai_structured(test_df, "gpt-4.1-mini", input_mode, file_cache_path)
print(result.iloc[8])
result = alpaca_df_to_qwen_messages(
    result, 
    output="content",
    #system_prompt="You are a meticulous assistant that writes formal specifications and verification code for Cryptol programs.", 
    drop_empty_output=False, 
    include_filename_in_user=False
    )
print(result.iloc[0])
for idx, row in result.iterrows():
    print(f"--- Result {idx + 1} ---")
    print(row["messages"])
    print()


filename           saw-script/intTests/test_llvm_x86_08/test.saw
filetype                                                     saw
instruction    Write a SAWScript verification harness for the...
input                                               test.bc test
output                                                          
content        enable_experimental;\n\nm <- llvm_load_module ...
Name: 8, dtype: object
messages    [{'role': 'system', 'content': 'You write SAWS...
filename                saw-script/intTests/test2045/test.saw
filetype                                                  saw
Name: 0, dtype: object
--- Result 1 ---
[{'role': 'system', 'content': 'You write SAWScript verification harnesses ONLY.\n\nOutput requirements:\n- Return exactly ONE fenced code block labeled `saw` and nothing else (no prose before/after).\n- No comments, no Markdown inside the code, ASCII only.\n- The code must load and parse in SAW.\n\nBehavioral rules:\n- Infer backend from the user context: us

In [6]:

'''
print(f"""\
Instruction: {result.instruction}
Input: {result.input}
Output: {result.output}
""")
'''

'\nprint(f"""Instruction: {result.instruction}\nInput: {result.input}\nOutput: {result.output}\n""")\n'

In [7]:
#result.to_json("temp/alpaca_instruct_sft_samples2.jsonl", lines=True, orient="records")